# Experiment 11
In this experiment, we're back to adversarial training, but trying the thresholding method. Judging the points using the loss from the validation set as a threshold after multiplying with a certain factor

In [1]:
!ls ../../Projects/data/

 MNIST			       SWaT_Dataset_Normal_v1.xlsx
 SWaT_Dataset_Attack_v0.csv    WADI_14days_new.csv
 SWaT_Dataset_Attack_v0.xlsx  'WADI.A1_9 Oct 2017'
 SWaT_Dataset_Normal_v0.csv   'WADI.A2_19 Nov 2019'
 SWaT_Dataset_Normal_v0.xlsx   WADI_attackdataLABLE.csv
 SWaT_Dataset_Normal_v1.csv


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch.optim import Adam
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from layers import *

from sklearn.metrics import confusion_matrix, f1_score, roc_curve, roc_auc_score, precision_score, recall_score

### Dataset:

In [2]:
df = pd.read_csv('../../Projects/data/SWaT_Dataset_Normal_v1.csv')
df = df.drop(columns=[' Timestamp', 'Normal/Attack'])
df = df.astype('float64')
mm = StandardScaler()
Normalized = pd.DataFrame(mm.fit_transform(df))
train_set = Normalized[: int(0.8 * Normalized.shape[0])]
validation_set = Normalized[int(0.8 * Normalized.shape[0]):]

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Dataloader:

In [4]:
window_size = 100

train_dataset = SWat_dataset(train_set, train_set, window_size, device)
validation_dataset = SWat_dataset(validation_set, validation_set, window_size, device)

batch_size = 4096
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

### Model:

In [5]:
class GEncoder(nn.Module):
    def __init__(self, num_nodes, window_size, alpha, k, device):
        super(GEncoder, self).__init__()
        self.num_nodes = num_nodes
        self.window_size = window_size
        self.conv1 = GCNLayer(window_size, 33)
        self.conv2 = GCNLayer(33, 10)
        self.idx = torch.arange(num_nodes).to(device)
        self.A = Graph_Directed_A(num_nodes, window_size, alpha, k, device)
        self.device = device

    def forward(self, X):
        X = torch.transpose(X, 1, 2)
        adj = self.A(self.idx)
        h = self.conv1(adj, X).relu()
        h = self.conv2(adj, h).relu()
        return h
        
        

class GCN_autoencoder(nn.Module):
    def __init__(self, encoder, num_nodes, window_size):
        super(GCN_autoencoder, self).__init__()
        self.window_size = window_size
        self.num_nodes = num_nodes
        self.encoder = encoder
        self.decoder = nn.Sequential(
            nn.Linear(510, 1200),
            nn.ReLU(),
            nn.Linear(1200, num_nodes * window_size)
        )
        
    def forward(self, X):
        x = self.encoder(X)
        x = torch.flatten(x,1)
        x = self.decoder(x)
        return x.view(-1, self.window_size, self.num_nodes)

    def get_adj(self):
        return self.encoder.A(self.encoder.idx)

In [6]:
num_nodes = 51 # number of nodes
alpha = 0.1 # hyperparameter for weights of edges
k = None  # max number of edges for each node
epochs = 200
out_channels = 2 # number of process states

In [7]:
Encoder = GEncoder(num_nodes, window_size, alpha, k, device)
AE1 = GCN_autoencoder(Encoder, num_nodes, window_size)
AE2 = GCN_autoencoder(Encoder, num_nodes, window_size)

In [8]:
AE1.to(device)
AE2.to(device)

GCN_autoencoder(
  (encoder): GEncoder(
    (conv1): GCNLayer(
      (dense): Linear(in_features=100, out_features=33, bias=True)
    )
    (conv2): GCNLayer(
      (dense): Linear(in_features=33, out_features=10, bias=True)
    )
    (A): Graph_ReLu_W()
  )
  (decoder): Sequential(
    (0): Linear(in_features=510, out_features=1200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1200, out_features=2500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=2500, out_features=5100, bias=True)
  )
)

In [9]:
optimizer1 = torch.optim.Adam(AE1.parameters())
optimizer2 = torch.optim.Adam(AE2.parameters())

In [10]:
# from torch.utils.tensorboard import SummaryWriter

In [11]:
AE1_val_history = []
AE2_val_history = []
for i in range(epochs):
    running_loss_AE1 = []
    running_loss_AE2 = []
    val_loss_AE1 = []
    val_loss_AE2 = []
    for index_b, features in enumerate(train_loader):
        
        w1 = AE1(features)
        w2 = AE2(features)
        w3 = AE2(w1)
        lossAE1 = (1 / (i + 1)) * torch.mean((features - w1) ** 2) + (1 - (1 / (i + 1))) * torch.mean((features - w3) ** 2)
        
        running_loss_AE1.append(lossAE1)
        lossAE1.backward()
        optimizer1.step()
        optimizer1.zero_grad()
        
        w1 = AE1(features)
        w2 = AE2(features)
        w3 = AE2(w1)
        lossAE2 = (1 / (i + 1)) * torch.mean((features - w2) ** 2) - (1 - (1 / (i + 1))) * torch.mean((features - w3) ** 2)
        
        running_loss_AE2.append(lossAE2)
        lossAE2.backward()
        optimizer2.step()
        optimizer2.zero_grad()
    
    for index_b, features in enumerate(validation_loader):
        with torch.no_grad():

            w1 = AE1(features)
            w2 = AE2(features)
            w3 = AE2(w1)
            lossAE1 = (1 / (i + 1)) * torch.mean((features - w1) ** 2) + (1 - (1 / (i + 1))) * torch.mean((features - w3) ** 2)
            lossAE2 = (1 / (i + 1)) * torch.mean((features - w2) ** 2) - (1 - (1 / (i + 1))) * torch.mean((features - w3) ** 2)
            val_loss_AE1.append(lossAE1)
            val_loss_AE2.append(lossAE2)
    AE1_val_history.append(torch.stack(val_loss_AE1).mean().item())
    AE2_val_history.append(torch.stack(val_loss_AE2).mean().item())
    print(f'Epoch: {i} ---> Val loss: AE1 {AE1_val_history[-1]:.4f}, AE2: {AE2_val_history[-1]:.4f}')
    print(f'Train loss: AE1 {torch.stack(running_loss_AE1).mean().item():.4f}, AE2 {torch.stack(running_loss_AE2).mean().item():.4f}')

Epoch: 0 ---> Val loss: AE1 0.1132, AE2: 0.1141
Train loss: AE1 0.2740, AE2 0.2647
Epoch: 1 ---> Val loss: AE1 0.1061, AE2: -0.0136
Train loss: AE1 0.2505, AE2 -0.1368
Epoch: 2 ---> Val loss: AE1 0.0971, AE2: -0.0460
Train loss: AE1 0.2708, AE2 -0.2196
Epoch: 3 ---> Val loss: AE1 0.0906, AE2: -0.0573
Train loss: AE1 0.2858, AE2 -0.2538
Epoch: 4 ---> Val loss: AE1 0.1704, AE2: -0.1213
Train loss: AE1 0.4687, AE2 -0.3160
Epoch: 5 ---> Val loss: AE1 0.1218, AE2: -0.0927
Train loss: AE1 0.3862, AE2 -0.3407
Epoch: 6 ---> Val loss: AE1 0.1014, AE2: -0.0817
Train loss: AE1 0.3645, AE2 -0.3408
Epoch: 7 ---> Val loss: AE1 0.1100, AE2: -0.0928
Train loss: AE1 0.3682, AE2 -0.3493
Epoch: 8 ---> Val loss: AE1 0.1055, AE2: -0.0910
Train loss: AE1 0.3710, AE2 -0.3564
Epoch: 9 ---> Val loss: AE1 0.1197, AE2: -0.1056
Train loss: AE1 0.3863, AE2 -0.3705
Epoch: 10 ---> Val loss: AE1 0.1850, AE2: -0.1608
Train loss: AE1 0.4085, AE2 -0.3793
Epoch: 11 ---> Val loss: AE1 0.1399, AE2: -0.1249
Train loss: AE1 

# Testing

In [31]:
df2 = pd.read_csv('../../Projects/data/SWaT_Dataset_Attack_v0.csv')
labels = df2['Normal/Attack']
df2 = df2.drop(columns=[' Timestamp', 'Normal/Attack'])
df2 = df2.astype('float64')
df2.columns = df.columns
test_normalized = pd.DataFrame(mm.transform(df2))

In [32]:
test_dataset = SWat_dataset(test_normalized, test_normalized, window_size, device)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [33]:
def testing(test_loader, alpha=.5, beta=.5):
    results=[]
    with torch.no_grad():
        for batch in test_loader:
            w1=AE1(batch)
            w2=AE2(w1)
            results.append(alpha * torch.mean((batch-w1)**2, axis=1) +
                           beta * torch.mean((batch-w2)**2, axis=1))
    return results

def get_threshold(val_loader):
    recon_errors = testing(val_loader)
    complete_vals = np.concatenate([torch.stack(recon_errors[:-1]).view(-1, num_nodes).detach().cpu().numpy(),
                                    recon_errors[-1].view(-1, num_nodes).detach().cpu().numpy()])
    return np.max(complete_vals, axis=0)

In [34]:
labels = labels.values
labels = [0 if (lab == 'Normal') else 1 for lab in labels]
windows_labels=[]
for i in range(len(labels)-window_size):
    windows_labels.append(list(np.int32(labels[i:i+window_size])))

In [35]:
y_test = [1.0 if (np.sum(window) > 0) else 0 for window in windows_labels]

In [36]:
thresholds = get_threshold(validation_loader)
results=testing(test_loader, alpha=0.3, beta=0.7)
y_pred=np.concatenate([torch.stack(results[:-1]).view(-1, num_nodes).detach().cpu().numpy(), 
                       results[-1].view(-1, num_nodes).detach().cpu().numpy()])

In [37]:
factor = 1

In [55]:
factor = 230
res = y_pred > (thresholds * factor)
boo = np.any(res, axis = 1)
verdicts = [1 if elem else 0 for elem in boo]
conf_matrix = confusion_matrix(y_test, verdicts)
TP = conf_matrix[1, 1]
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
F1 = f1_score(y_test, verdicts)
precision = precision_score(y_test, verdicts)
recall = recall_score(y_test, verdicts)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", F1)
print(f'TP: {TP}\nTN: {TN}\nFP: {FP}\nFN: {FN}')
print(f'Factor is: {factor}')

Precision: 0.784568523469343
Recall: 0.617271818213165
F1 Score: 0.6909375120614458
TP: 35803
TN: 381986
FP: 9831
FN: 22199
Factor is: 230


In [37]:
torch.save(AE1.state_dict(), 'GAE_77_Relu_50_epochs.pth')

In [20]:
AE1.load_state_dict(torch.load('GAE_77_Relu_50_epochs.pth'))

<All keys matched successfully>